# Threshold Tuning Showcase

This notebook demonstrates tuning of the rule-based detection system across multiple configurations (strict, moderate, relaxed). It uses the KeyRecs dataset to simulate keystroke-enhanced login attempts.

### Setup

This cell prepares the working environment by importing required libraries and ensuring that all necessary directories for data, results, and models exist.

In [ ]:
# If running in Colab, install required packages
# !pip install seaborn matplotlib scikit-learn pandas joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import random
import ast
from datetime import datetime, timedelta

os.makedirs("data", exist_ok=True)
os.makedirs("results", exist_ok=True)
os.makedirs("models", exist_ok=True)
os.makedirs("keystroke_ml/processed", exist_ok=True)


### Load and Prepare Keystroke Data

This step loads the KeyRecs dataset, assigns binary labels ('o = normal user', '1 = attacker'), and extract relevant timing features.

In [ ]:
# Load and process KeyRecs dataset
df = pd.read_csv("data/fixed-text.csv")
known_ids = [f'p{str(i).zfill(3)}' for i in range(1, 11)] 
known = df[df['participant'].isin(known_ids)].copy()
unknown = df[~df['participant'].isin(known_ids)].copy()
known['label'] = 0
unknown['label'] = 1
combined = pd.concat([known, unknown])

# Extract timing features
feature_cols = combined.columns[4:-1]
X = combined[feature_cols].copy()
X['label'] = combined['label'].values
X.to_csv("keystroke_ml/processed/keystroke_vectors.csv", index=False)
print(f"Processed {X.shape[0]} samples.")


### Simulate Login Activity with Keystroke Context

This section generates a synthetic login dataset simulating different user behaviors: 
- **Normal users** suceed quickly.
- **Brute force attackers** fail rapidly with repeated attempts.
- **Evasive attackers** use slower, varied patterns.

Each login attempt includes a keystroke vector sampled from the KeyRecs dataset to maintain realism in timing features.

In [ ]:
keystroke_df = pd.read_csv("keystroke_ml/processed/keystroke_vectors.csv")
known_vectors = keystroke_df[keystroke_df["label"] == 0].drop(columns=["label"]).values.tolist()
unknown_vectors = keystroke_df[keystroke_df["label"] == 1].drop(columns=["label"]).values.tolist()

def simulate_login_attempts(user_type, user_id, outcomes, min_delay, max_delay, varied_usernames=False, stop_on_success=False, keystroke_vector=None):
    logs = []
    last_time = datetime.now()
    session_id = f"{user_type}_{user_id}"

    for attempt, result in enumerate(outcomes):
        delay_secs = random.randint(min_delay, max_delay)
        delay = timedelta(seconds=delay_secs)
        current_time = last_time + delay
        last_time = current_time
        username = f"{user_type}_user_{user_id}" if not varied_usernames else f"{user_type}_rand_{random.randint(0,999)}"
        logs.append({
            "timestamp": current_time.isoformat(),
            "username": username,
            "result": result,
            "user_type": user_type,
            "session_id": session_id,
            "attempt_number": attempt + 1,
            "time_since_last": 0 if attempt == 0 else delay.total_seconds(),
            "keystroke_vector": keystroke_vector,
            "label": 0 if user_type == "normal" else 1
        })
        if result == "success" and stop_on_success:
            break
    return logs

# Generate login simulation
def generate_login_simulation():
    all_logs = []

    for i in range(100):
        outcomes = ["failure", "success"]
        vector = random.choice(known_vectors)
        logs = simulate_login_attempts("normal", i, outcomes, 3, 5, keystroke_vector=vector)
        all_logs.extend(logs)

    for i in range(30):
        outcomes = ["failure"] * 20
        vector = random.choice(unknown_vectors)
        logs = simulate_login_attempts("brute", i, outcomes, 0, 1, True, True, vector)
        all_logs.extend(logs)

    for i in range(30):
        outcomes = ["failure"] * 5 + ["success"]
        vector = random.choice(unknown_vectors)
        logs = simulate_login_attempts("evasive", i, outcomes, 3, 15, True, True, vector)
        all_logs.extend(logs)

    df = pd.DataFrame(all_logs)
    df["keystroke_vector"] = df["keystroke_vector"].apply(str)
    df.to_csv("data/login_simulation.csv", index=False)
    print(f"Simulated {len(df)} login attempts.")

generate_login_simulation()


### Apply Rule-Based Detection with Varying Threshold Profiles

This block evaluates detection performances across three policy configurations:
- **Strict**, **Moderate**, and **Relaxed**

Each profile modifies thresholds for detecting brute-force behavior,evasive behavior, and anomalous typing patterns based on keystroke timing (average delay and variance).

Performance is measures using precision, recall, and F1 score to compare the tradeoffs of each configuration.


In [ ]:
# Parse keystroke_vector back into list format
df = pd.read_csv("data/login_simulation.csv")
df["keystroke_vector"] = df["keystroke_vector"].apply(ast.literal_eval)

# Define threshold profiles
threshold_profiles = {
    "Strict": {
        "BRUTE_FORCE_ATTEMPT_THRESHOLD": 5,
        "BRUTE_FORCE_TIME_THRESHOLD": 2,
        "EVASIVE_DURATION_THRESHOLD": 30,
        "EVASIVE_USERNAME_THRESHOLD": 2,
        "HIGH_VARIANCE_THRESHOLD": 0.3,
        "LOW_TYPING_TIME_THRESHOLD": 0.1
    },
    "Moderate": {
        "BRUTE_FORCE_ATTEMPT_THRESHOLD": 10,
        "BRUTE_FORCE_TIME_THRESHOLD": 3,
        "EVASIVE_DURATION_THRESHOLD": 60,
        "EVASIVE_USERNAME_THRESHOLD": 3,
        "HIGH_VARIANCE_THRESHOLD": 0.45,
        "LOW_TYPING_TIME_THRESHOLD": 0.08
    },
    "Relaxed": {
        "BRUTE_FORCE_ATTEMPT_THRESHOLD": 20,
        "BRUTE_FORCE_TIME_THRESHOLD": 5,
        "EVASIVE_DURATION_THRESHOLD": 90,
        "EVASIVE_USERNAME_THRESHOLD": 5,
        "HIGH_VARIANCE_THRESHOLD": 0.6,
        "LOW_TYPING_TIME_THRESHOLD": 0.05
    }
}

from sklearn.metrics import precision_recall_fscore_support

results = []

for profile, thres in threshold_profiles.items():
    session_preds = []
    for session_id, group in df.groupby("session_id"):
        attempts = len(group)
        total_time = group["time_since_last"].sum()
        usernames = group["username"].nunique()
        success_rate = (group["result"] == "success").sum() / attempts if attempts else 0
        vector = [v for sublist in group["keystroke_vector"].tolist() for v in sublist]
        avg_delay = np.mean(vector) if vector else 0
        std_delay = np.std(vector) if vector else 0

        rule_flag = 0
        if attempts >= thres["BRUTE_FORCE_ATTEMPT_THRESHOLD"] and total_time <= thres["BRUTE_FORCE_TIME_THRESHOLD"]:
            rule_flag = 1
        elif success_rate < 0.1 and attempts >= 5 and usernames >= thres["EVASIVE_USERNAME_THRESHOLD"] and total_time > thres["EVASIVE_DURATION_THRESHOLD"]:
            rule_flag = 1
        elif std_delay > thres["HIGH_VARIANCE_THRESHOLD"]:
            rule_flag = 1
        elif avg_delay < thres["LOW_TYPING_TIME_THRESHOLD"]:
            rule_flag = 1

        session_preds.append((group["label"].iloc[0], rule_flag))

    y_true, y_pred = zip(*session_preds)
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary', zero_division=0)
    results.append({
        "Profile": profile,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1
    })

df_eval = pd.DataFrame(results)


### Visualize Detection Tradeoffs Across Thresholds

This chart compares the rule-based system's precision, recall, and F11 score across all threshold profiles. It highlights how stricter rules catch more attacks (higher recall) but may increase false positives (lower precision), while relaxed profiles reduce false alarms but risk missing threats.

Note: Rule-based only

In [ ]:
# Visualization
sns.set(style="whitegrid")
melted = df_eval.melt(id_vars="Profile", var_name="Metric", value_name="Score")
plt.figure(figsize=(10, 5))
sns.barplot(data=melted, x="Metric", y="Score", hue="Profile")
plt.title("Rule-Based Detection Performance Across Threshold Profiles")
plt.ylim(0, 1.05)
plt.tight_layout()
plt.show()